---

## 1. Recommended Setup (sitecustomize.py) {#recommended-setup}

**Best practice:** Install sitecustomize.py once per cluster for automatic patching.

### Why This Is Necessary

Databricks loads its import hooks during Python startup, **before** any notebook code runs. The only reliable way to fix editable installs is to patch **during** Python initialization using `sitecustomize.py`.


### Step 1: Install sitecustomize.py (Run ONCE per cluster)


In [ ]:
from dbx_patch import install_sitecustomize

# Install sitecustomize.py to auto-apply patches on Python startup
install_sitecustomize(verbose=True)

### Step 2: Python Will Restart Automatically

✅ **When running in Databricks:** Python kernel will restart automatically after installation!

⚠️ **If not in Databricks:** You'll need to manually restart the kernel (see cell below).


In [ ]:
# Manual restart (only needed if not running in Databricks)
# Uncomment if needed:

# import os
# os._exit(0)  # Kernel will automatically restart

---

## 2. Verify Installation {#verification}

Run these cells after the kernel restarts to confirm everything is working.


### Check sitecustomize.py Status


In [ ]:
from dbx_patch import check_sitecustomize_status

# Verify sitecustomize.py is installed and active
status = check_sitecustomize_status(verbose=True)

if status.is_dbx_patch:
    print("\n✅ sitecustomize.py is installed and active!")
else:
    print("\n❌ sitecustomize.py not properly installed - see troubleshooting")

### Verify Patches Are Applied


In [ ]:
from dbx_patch import verify_editable_installs

# Check that patches are applied and editable installs are detected
verify_editable_installs(verbose=True)

---

## 3. Using Editable Packages {#usage}

After sitecustomize.py is installed, editable installs work automatically!


### Install an Editable Package


In [ ]:
# Install your package in editable mode
# Uncomment and modify the path:

# %pip install -e /Workspace/Repos/your-repo/your-package

# Or from a local path:
# %pip install -e /path/to/your/package

print("📦 Install your editable package above, then continue...")

### Import and Use Your Package


In [ ]:
# Just import - no manual patching needed!
# import your_package

# Use it:
# result = your_package.some_function()
# print(result)

print("✅ Editable imports work automatically!")

---

## 4. Development Workflow with Autoreload {#development}

For active development, enable autoreload to automatically reload modules when they change.


### Enable Autoreload


In [ ]:
%load_ext autoreload
%autoreload 2

print("✅ Autoreload enabled - modules will reload automatically when changed")

### Complete Development Example


In [ ]:
# 1. Install your package in editable mode
# %pip install -e /Workspace/Repos/my-repo/my-package

# 2. Import it
# import my_package

# 3. Use it
# result = my_package.my_function()
# print(result)

# 4. Edit your source code in Repos
# 5. Re-run this cell - changes are automatically loaded!

print("🔄 Edit source code → Re-run cell → See changes instantly")

---

## 5. Alternative: Manual Patching {#manual}

If you can't use sitecustomize.py, you can manually apply patches in each notebook.

**⚠️ Limitations:**

- Must be called in every notebook
- Timing issues with early imports
- Not recommended for production


### Apply Patches Manually


In [ ]:
from dbx_patch import patch_dbx

# Apply all patches with verbose output
result = patch_dbx(force_refresh=False)

print(f"\n✅ Patches applied: {result.overall_success}")
print(f"📦 Editable paths found: {len(result.editable_paths)}")

### After Installing New Packages

When you install a new editable package, refresh the detection:


In [ ]:
from dbx_patch.pth_processor import process_all_pth_files

# After: %pip install -e /path/to/new-package
# Refresh to detect the new package
process_all_pth_files(force=True, verbose=True)

---

## 6. Troubleshooting {#troubleshooting}


### Issue: Still getting ModuleNotFoundError


In [ ]:
# Check if patches are applied
from dbx_patch import check_patch_status

status = check_patch_status(verbose=True)

if not status.wsfs_hook_patched:
    print("\n⚠️ Import hook not patched - try restarting kernel")

### Issue: Want to see diagnostic information


In [ ]:
import sys

from dbx_patch.pth_processor import get_editable_install_paths, get_site_packages_dirs

print("=== Diagnostic Information ===\n")

# Site-packages directories
site_dirs = get_site_packages_dirs()
print(f"Site-packages directories: {len(site_dirs)}")
for d in site_dirs:
    print(f"  • {d}")

# Editable paths
editable_paths = get_editable_install_paths()
print(f"\nEditable install paths: {len(editable_paths)}")
for path in sorted(editable_paths):
    in_path = "✅" if path in sys.path else "❌"
    print(f"  {in_path} {path}")

### Issue: Need to uninstall sitecustomize.py


In [ ]:
from dbx_patch import uninstall_sitecustomize

# Remove sitecustomize.py
uninstall_sitecustomize(verbose=True)

# Restart kernel for changes to take effect
print("\n⚠️ Restart kernel to complete uninstallation")

---

## 7. Advanced Examples {#advanced}


### Selective Patching

Apply only specific patches if needed:


In [ ]:
from dbx_patch.patches.python_path_hook_patch import PythonPathHookPatch
from dbx_patch.patches.wsfs_import_hook_patch import WsfsImportHookPatch
from dbx_patch.pth_processor import process_all_pth_files

# Apply only .pth file processing
process_all_pth_files(verbose=True)

# Apply specific patches using class-based API
WsfsImportHookPatch().patch()
PythonPathHookPatch().patch()

### Check What's Detected Without Modifying


In [ ]:
from dbx_patch.pth_processor import get_editable_install_paths

# Get editable paths without modifying sys.path
editable_paths = get_editable_install_paths()

print(f"Found {len(editable_paths)} editable install(s):")
for path in sorted(editable_paths):
    print(f"  • {path}")

### Multiple Editable Packages


In [ ]:
# Install multiple packages
# %pip install -e /Workspace/Repos/repo1/package1
# %pip install -e /Workspace/Repos/repo2/package2
# %pip install -e /Workspace/Users/me/dev/package3

# Import all
# import package1
# import package2
# import package3

print("✅ Multiple editable packages supported")

### Remove All Patches (Restore Original Behavior)


In [ ]:
from dbx_patch import remove_all_patches

# Remove all patches
result = remove_all_patches(verbose=True)

if result.success:
    print("\n✅ Patches removed - original Databricks behavior restored")

---

## Cluster Init Script (Production)

For production deployments, add this to your cluster init script to automatically install sitecustomize.py on cluster startup:

```bash
#!/bin/bash

# Install dbx-patch
pip install dbx-patch

# Install sitecustomize.py for auto-patching
python3 << 'EOF'
from dbx_patch import install_sitecustomize
install_sitecustomize(verbose=False)
EOF
```

After this, editable installs work automatically for all users on the cluster.

---

## Summary

✅ **One-time setup:** `install_sitecustomize()` + restart kernel  
✅ **Automatic:** Patches apply on Python startup  
✅ **No manual calls:** Just import your editable packages  
✅ **Cluster-wide:** Works for all users/notebooks  
✅ **Production-ready:** Add to init script for automatic setup

---

## Resources

- [Getting Started Guide](../docs/docs/files/getting-started.md) - Complete installation and usage guide
- [Technical Implementation](../docs/docs/files/implementation.md) - Architecture and technical details
- [GitHub Repository](https://github.com/twsl/dbx-patch) - Source code and issues
